# Data Cleaning Notebook

In [7]:
import pandas as pd

In [9]:
df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-records.csv")
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Place,Place.1,Conference,Conference.1,Division,Division.1,Division.2,...,All-Star.1,Margin,Margin.1,Month,Month.1,Month.2,Month.3,Month.4,Month.5,Month.6
Rk,Team,Overall,Home,Road,E,W,A,C,SE,NW,...,≤3,≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr,Year
1,Oklahoma City Thunder,68-14,36-6,32-8,29-1,39-13,10-0,9-1,10-0,12-4,...,1-4,54-5,4-0,11-4,12-1,10-4,11-2,15-1,5-2,2025
2,Cleveland Cavaliers,64-18,34-7,30-11,41-11,23-7,16-3,12-4,13-4,9-1,...,5-4,41-7,5-0,12-3,12-1,10-5,10-1,11-5,4-3,2025
3,Boston Celtics,61-21,28-13,33-8,39-13,22-8,14-2,12-6,13-5,8-2,...,8-3,41-7,4-1,12-2,8-6,10-6,8-3,14-1,5-2,2025
4,Houston Rockets,52-30,29-12,23-18,21-9,31-21,6-4,7-3,8-2,9-9,...,8-5,26-12,3-2,11-4,7-5,11-4,5-7,12-5,3-3,2025
5,New York Knicks,51-31,27-14,24-17,34-18,17-13,12-4,7-10,15-4,6-4,...,7-3,31-19,2-2,9-6,12-2,9-6,7-4,8-7,4-4,2025
6,Denver Nuggets,50-32,26-15,24-17,18-12,32-20,6-4,4-6,8-2,8-8,...,8-6,31-17,2-2,8-5,8-6,11-6,10-2,8-7,3-4,2025
7,Indiana Pacers,50-32,29-12,21-20,29-22,21-10,10-7,10-6,9-9,6-4,...,6-2,25-17,2-3,7-8,7-7,10-2,7-5,11-6,6-1,2025
8,Los Angeles Clippers,50-32,30-11,20-21,21-9,29-23,7-3,4-6,10-0,8-10,...,5-7,32-13,2-3,10-6,7-5,9-6,4-7,11-5,7-0,2025
9,Los Angeles Lakers,50-32,31-10,19-22,14-16,36-16,7-3,2-8,5-5,11-8,...,4-6,22-21,3-2,8-6,7-6,9-5,10-2,9-8,4-3,2025


In [20]:
team_stats_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats.csv")
team_stats_df

,Season,Team,GP,W,L,WIN%,Min,PTS,FGM,FGA,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PLUS_MINUS
0,2025,Oklahoma City Thunder,82,68,14,0.829,48.1,120.5,44.6,92.7,...,34.2,44.8,26.9,11.7,10.3,5.7,4.8,19.9,18.1,12.9
1,2025,Cleveland Cavaliers,82,64,18,0.780,48.2,121.9,44.5,90.8,...,34.2,45.4,28.1,13.2,8.2,4.3,4.4,18.1,18.9,9.5
2,2025,Boston Celtics,82,61,21,0.744,48.4,116.3,41.6,90.0,...,33.9,45.3,26.1,11.9,7.2,5.5,3.6,15.9,17.5,9.1
3,2025,Houston Rockets,82,52,30,0.634,48.2,114.3,42.5,93.4,...,33.8,48.5,23.3,13.9,8.4,5.0,5.7,19.0,19.2,4.5
4,2025,New York Knicks,82,51,31,0.622,48.5,115.8,43.3,89.2,...,31.8,42.6,27.5,13.3,8.2,4.0,5.0,17.2,19.0,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2023,Portland Trail Blazers,82,33,49,0.402,48.1,113.4,40.5,85.4,...,31.1,40.5,24.2,14.5,6.7,4.6,4.3,20.0,20.5,-4.0
85,2023,Charlotte Hornets,82,27,55,0.329,48.4,111.0,41.3,90.4,...,33.5,44.5,25.1,14.2,7.7,5.2,5.7,20.3,20.3,-6.2
86,2023,Houston Rockets,82,22,60,0.268,48.2,110.7,40.6,88.9,...,32.9,46.3,22.4,16.2,7.3,4.6,6.2,20.5,20.6,-7.9
87,2023,San Antonio Spurs,82,22,60,0.268,48.4,113.0,43.1,92.6,...,31.9,43.7,27.2,15.3,7.0,3.9,5.3,19.9,18.5,-10.0
